In [96]:
def check_unique_correspondence(df, col1, col2):
    """Checks if each unique value in col1 corresponds to a unique value in col2."""

    unique_col1 = df[col1].unique()
    unique_col2_per_col1 = df.groupby(col1)[col2].unique()

    return all(len(values) == 1 for values in unique_col2_per_col1.values)

## Import libraries

In [97]:
import gzip
import shutil
import os

import pandas as pd
import numpy as np
import re

import itertools


## Load data

In [98]:
df_receipts = pd.read_json('receipts.json',lines=True)
df_brands = pd.read_json('brands.json',lines=True)
df_users = pd.read_json('users.json',lines=True,encoding = 'utf-8-sig')

In [99]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [100]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [101]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [102]:
# Renaming all id columns for each dataframe to avoid errors and confusion
df_users.rename(columns={'_id':'user_id'}, inplace=True)
df_brands.rename(columns={'_id':'brand_id'}, inplace=True)
df_receipts.rename(columns={'_id':'receipt_id'}, inplace=True)

## Users data

In [103]:
df_users

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


In [104]:
# Extracting user_id, createdDate, and lastLogin in clean format
df_users['user_id'] = df_users['user_id'].apply(lambda x: x.get('$oid'))
df_users['createdDate'] = df_users['createdDate'].apply(lambda x: x.get('$date') if str(x) != str(np.nan) else x)
df_users['lastLogin'] = df_users['lastLogin'].apply(lambda x: x.get('$date') if str(x) != str(np.nan) else x)

df_users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [105]:
# Converting createdDate and lastLogin columns to datetime format from milliseconds
df_users['createdDate'] = pd.to_datetime(df_users['createdDate']//1000, unit='s') # convert to datetime
df_users['lastLogin'] = pd.to_datetime(df_users['lastLogin']//1000, unit='s')

df_users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


In [106]:
# Sorting users data by user id
df_users.sort_values(by='user_id', inplace=True)

df_users

,user_id,active,createdDate,lastLogin,role,signUpSource,state
494,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
475,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
476,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
477,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
...,...,...,...,...,...,...,...
417,60255883efa60114d20e5d4e,True,2021-02-11 16:17:07,2021-02-11 16:18:29,consumer,Email,WI
418,602558adb5459313e1e9b7ce,True,2021-02-11 16:17:49,2021-02-11 16:17:49,consumer,Email,WI
415,602558b1efa60114d20e5dc7,True,2021-02-11 16:17:53,2021-02-11 16:17:53,consumer,Email,WI
427,60268c69efa6011bb151075f,True,2021-02-12 14:10:49,2021-02-12 14:12:07,consumer,Email,WI


In [107]:
# Viewing all duplicate rows
df_users[df_users.duplicated(keep=False)]

,user_id,active,createdDate,lastLogin,role,signUpSource,state
494,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
475,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
476,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
477,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
...,...,...,...,...,...,...,...
365,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04,2021-02-02 00:28:04,consumer,Email,WI
387,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53,2021-02-04 17:20:30,consumer,Email,WI
385,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53,2021-02-04 17:20:30,consumer,Email,WI
393,60229990b57b8a12187fe9e0,True,2021-02-09 14:17:52,2021-02-09 14:17:52,consumer,Email,WI


In [108]:
# Out of the above 353 duplicate rows, the following 283 rows will be removed in the next step
# This means that 353-283= 70 rows have duplicate occurences
df_users[df_users.duplicated()]

,user_id,active,createdDate,lastLogin,role,signUpSource,state
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
475,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
476,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
477,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
478,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
...,...,...,...,...,...,...,...
374,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04,2021-02-02 00:28:04,consumer,Email,WI
373,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04,2021-02-02 00:28:04,consumer,Email,WI
365,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04,2021-02-02 00:28:04,consumer,Email,WI
385,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53,2021-02-04 17:20:30,consumer,Email,WI


In [109]:
# Number of rows expected after duplicates are removed
df_users.shape[0]-353+70

212

In [110]:
# Dropping duplicate rows and keeping only one occurence of each
df_users.drop_duplicates(keep='first', inplace=True)

df_users

,user_id,active,createdDate,lastLogin,role,signUpSource,state
494,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
388,55308179e4b0eabd8f99caa2,True,2015-04-17 03:43:53,2018-05-07 17:23:40,consumer,NaN,WI
462,5964eb07e4b03efd0c0f267b,True,2017-07-11 15:13:11,2021-03-04 19:07:49,fetch-staff,NaN,IL
407,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54,2021-02-08 16:42:58,fetch-staff,NaN,WI
430,5a43c08fe4b014fd6b6a0612,True,2017-12-27 15:47:27,2021-02-12 16:22:37,consumer,NaN,NaN
...,...,...,...,...,...,...,...
417,60255883efa60114d20e5d4e,True,2021-02-11 16:17:07,2021-02-11 16:18:29,consumer,Email,WI
418,602558adb5459313e1e9b7ce,True,2021-02-11 16:17:49,2021-02-11 16:17:49,consumer,Email,WI
415,602558b1efa60114d20e5dc7,True,2021-02-11 16:17:53,2021-02-11 16:17:53,consumer,Email,WI
427,60268c69efa6011bb151075f,True,2021-02-12 14:10:49,2021-02-12 14:12:07,consumer,Email,WI


It can be seen that the resulting dataframe has 212 rows. Hence verified that the duplicate rows removal was executed correctly.

In [111]:
# Checking out information of the final users data that will be loaded to the database
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212 entries, 494 to 421
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       212 non-null    object        
 1   active        212 non-null    bool          
 2   createdDate   212 non-null    datetime64[ns]
 3   lastLogin     172 non-null    datetime64[ns]
 4   role          212 non-null    object        
 5   signUpSource  207 non-null    object        
 6   state         206 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 11.8+ KB


In [112]:
# Checking the number of NaN values in each column
df_users.isna().sum().sort_values(ascending=True)

,0
user_id,0
active,0
createdDate,0
role,0
signUpSource,5
state,6
lastLogin,40


It can be seen that a few values are missing for signUpSource and state of existing users.

lastLogin has not been recorded for 40 out of 212 users.

In [113]:
# Checking the unique values that 'active' column has
df_users['active'].unique()

array([ True, False])

The 'active' column has a boolean (True/False) value.

In [114]:
# Checking the unique values that 'role' column has
df_users['role'].unique()

array(['fetch-staff', 'consumer'], dtype=object)

The 'role' of a user can only have one of the two values: 'fetch-staff' or 'consumer'.

Hence, 'role' column can be modeled as an ENUM in the actual database.

In [115]:
# Checking the unique values that 'signUpSource' column has
df_users['signUpSource'].unique()

array([nan, 'Google', 'Email'], dtype=object)

The signUpSource can be NULL and currently has only two sources: 'Google' and 'Email'

In [116]:
# Checking the unique values that 'state' column has
df_users['state'].unique()

array([nan, 'WI', 'IL', 'AL', 'NH', 'KY', 'CO', 'OH', 'SC'], dtype=object)

Currently, the users table has users from only 8 states and for a few users, the state has not been recorded (may be optional for users to specify while creating a new user)

## Brands

In [117]:
df_brands

,brand_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


In [118]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand_id      1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


In [119]:
# Renaming barcode and name columns
df_brands.rename(columns={'name': 'brand_name'}, inplace=True)

df_brands.head()

,brand_id,barcode,category,categoryCode,cpg,brand_name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [120]:
# Extracting brand_id in clean format
df_brands['brand_id'] = df_brands['brand_id'].apply(lambda x: x.get('$oid'))

df_brands.head()

,brand_id,barcode,category,categoryCode,cpg,brand_name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [121]:
# Checking the number of NaN values in each column
df_brands.isna().sum().sort_values(ascending=True)

,0
brand_id,0
barcode,0
cpg,0
brand_name,0
category,155
brandCode,234
topBrand,612
categoryCode,650


In [122]:
# Converting cpg from nested dictionary into a dataframe
df_cpg = pd.DataFrame([x for x in df_brands['cpg']])

df_cpg

,$id,$ref
0,{'$oid': '601ac114be37ce2ead437550'},Cogs
1,{'$oid': '5332f5fbe4b03c9a25efd0ba'},Cogs
2,{'$oid': '601ac142be37ce2ead437559'},Cogs
3,{'$oid': '601ac142be37ce2ead437559'},Cogs
4,{'$oid': '5332fa12e4b03c9a25efd1e7'},Cogs
...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90bf'},Cogs
1163,{'$oid': '53e10d6368abd3c7065097cc'},Cogs
1164,{'$oid': '5332fa12e4b03c9a25efd1e7'},Cogs
1165,{'$oid': '5332f5f6e4b03c9a25efd0b4'},Cogs


In [123]:
# Renaming cpg columns
df_cpg.rename(columns={'$id': 'cpg_id', '$ref': 'cpg_ref'}, inplace=True)

# Extracting cpg_id in clean format
df_cpg['cpg_id'] = df_cpg['cpg_id'].apply(lambda x: x.get('$oid'))

df_cpg

,cpg_id,cpg_ref
0,601ac114be37ce2ead437550,Cogs
1,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead437559,Cogs
4,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...
1162,5f77274dbe37ce6b592e90bf,Cogs
1163,53e10d6368abd3c7065097cc,Cogs
1164,5332fa12e4b03c9a25efd1e7,Cogs
1165,5332f5f6e4b03c9a25efd0b4,Cogs


In [124]:
# Checking the number of NaN values among cpg columns
df_cpg.isna().sum().sort_values(ascending=True)

,0
cpg_id,0
cpg_ref,0


In [125]:
# Checking what are the unique values within cpg_ref
df_cpg['cpg_ref'].unique()

array(['Cogs', 'Cpgs'], dtype=object)

There are two unique values: 'Cpgs' and 'Cogs' within the cpg_ref column

In [126]:
# Number of unique values of cpg_id
df_cpg['cpg_id'].nunique()

196

In [127]:
print(check_unique_correspondence(df_cpg, 'cpg_id', 'cpg_ref'))

False


In [128]:
# Dropping cpg column in brands data and adding cpg_id and cpg_ref column
df_brands.drop('cpg', axis=1, inplace=True)
df_brands = pd.concat([df_brands, df_cpg], axis=1)

df_brands

,brand_id,barcode,category,categoryCode,brand_name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,5f77274dbe37ce6b592e90bf,Cogs
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,53e10d6368abd3c7065097cc,Cogs
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,5332fa12e4b03c9a25efd1e7,Cogs
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,5332f5f6e4b03c9a25efd0b4,Cogs


In [129]:
# Checking number ogf unique values in each column of brands table
df_brands.nunique().sort_values(ascending=False)

,0
brand_id,1167
barcode,1160
brand_name,1156
brandCode,897
cpg_id,196
category,23
categoryCode,14
topBrand,2
cpg_ref,2


Since there are 1167 unique brand_ids and the brands data has 1167 rows, there ideally should not be any duplicate rows in the brands data. Let's check that.

In [130]:
# Checking duplicates in brands data
df_brands[df_brands.duplicated()]

,brand_id,barcode,category,categoryCode,brand_name,topBrand,brandCode,cpg_id,cpg_ref


In [131]:
df_brands.isna().sum().sort_values(ascending=True)

,0
brand_id,0
barcode,0
brand_name,0
cpg_id,0
cpg_ref,0
category,155
brandCode,234
topBrand,612
categoryCode,650


In [132]:
# Checking unique values that 'topBrand' column can take
df_brands['topBrand'].unique()

array([ 0., nan,  1.])

The 'topBrand' column can be null and currently has float values 0 and 1. It can be likely converted to a boolean datatype.

##### DATA QUALITY ISSUE

In [133]:
df_brands[df_brands.duplicated('barcode', keep=False)].sort_values(by='barcode')

,brand_id,barcode,category,categoryCode,brand_name,topBrand,brandCode,cpg_id,cpg_ref
467,5c409ab4cd244a3539b84162,511111004790,Baking,NaN,alexa,1.0,ALEXA,55b62995e4b0d8e685c14213,Cogs
1071,5cdacd63166eb33eb7ce0fa8,511111004790,Condiments & Sauces,NaN,Bitten Dressing,NaN,BITTEN,559c2234e4b06aca36af13c6,Cogs
152,5c45f91b87ff3552f950f027,511111204923,Grocery,NaN,Brand1,1.0,0987654321,5c45f8b087ff3552f950f026,Cogs
536,5d6027f46d5f3b23d1bc7906,511111204923,Snacks,NaN,CHESTER'S,NaN,CHESTERS,5332f5fbe4b03c9a25efd0ba,Cogs
20,5c4699f387ff3577e203ea29,511111305125,Baby,NaN,Chris Image Test,NaN,CHRISIMAGE,55b62995e4b0d8e685c14213,Cogs
651,5d642d65a3a018514994f42d,511111305125,Magazines,NaN,Rachael Ray Everyday,NaN,511111305125,5d5d4fd16d5f3b23d1bc7905,Cogs
129,5a7e0604e4b0aedb3b84afd3,511111504139,Beverages,NaN,Chris Brand XYZ,NaN,CHRISXYZ,55b62995e4b0d8e685c14213,Cogs
299,5a8c33f3e4b07f0a2dac8943,511111504139,Grocery,NaN,Pace,0.0,PACE,5a734034e4b0d58f376be874,Cogs
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,NaN,test,NaN,TEST,59ba6f1ce4b092b29c167346,Cogs
412,5ccb2ece166eb31bbbadccbe,511111504788,Condiments & Sauces,NaN,The Pioneer Woman,NaN,PIONEER WOMAN,559c2234e4b06aca36af13c6,Cogs


## Normalizing brands table (to 3NF or 3rd Normal Form) and creating Category table

In [134]:
df_brands['category'].unique()

array(['Baking', 'Beverages', 'Candy & Sweets', 'Condiments & Sauces',
       'Canned Goods & Soups', nan, 'Magazines', 'Breakfast & Cereal',
       'Beer Wine Spirits', 'Health & Wellness', 'Beauty', 'Baby',
       'Frozen', 'Grocery', 'Snacks', 'Household', 'Personal Care',
       'Dairy', 'Cleaning & Home Improvement', 'Deli',
       'Beauty & Personal Care', 'Bread & Bakery', 'Outdoor',
       'Dairy & Refrigerated'], dtype=object)

In [135]:
df_brands['categoryCode'].unique()

array(['BAKING', 'BEVERAGES', 'CANDY_AND_SWEETS', nan,
       'HEALTHY_AND_WELLNESS', 'GROCERY', 'PERSONAL_CARE',
       'CLEANING_AND_HOME_IMPROVEMENT', 'BEER_WINE_SPIRITS', 'BABY',
       'BREAD_AND_BAKERY', 'OUTDOOR', 'DAIRY_AND_REFRIGERATED',
       'MAGAZINES', 'FROZEN'], dtype=object)

The following categories have a categoryCode existing in the current brands data:
- Baking -> BAKING
- Beverages -> BEVERAGES
- Candy & Sweets -> CANDY_AND_SWEETS
- Magazines -> MAGAZINES
- Beer Wine Spirits -> BEER_WINE_SPIRITS
- Health & Wellness -> HEALTHY_AND_WELLNESS
- Baby -> BABY
- Frozen -> FROZEN
- Grocery -> GROCERY
- Personal Care -> PERSONAL_CARE
- Cleaning & Home Improvement -> CLEANING_AND_HOME_IMPROVEMENT
- Bread & Bakery -> BREAD_AND_BAKERY
- Outdoor -> OUTDOOR
- Dairy & Refrigerated -> DAIRY_AND_REFRIGERATE

The following categories don't have categoryCodes yet:

- Condiments & Sauces
- Canned Goods & Soups
- Breakfast & Cereal
- Beauty
- Beauty & Personal Care
- Snacks
- Household
- Dairy
- Deli

This indicates a transitive dependecy bertween categoryCode and category

In [136]:
# Checking if category is unique for unique brand_id
print(check_unique_correspondence(df_brands, 'brand_id', 'category'))
print(check_unique_correspondence(df_brands, 'brand_id', 'categoryCode'))

print("\n")

# Checking if brand is unique for category
print(check_unique_correspondence(df_brands, 'category', 'brand_id'))
print(check_unique_correspondence(df_brands, 'categoryCode', 'brand_id'))

True
True


False
False


This indicates a one to many relationship between category and brand

In [137]:
# Creating the CATEGORY entity of the E-R diagram
df_category = df_brands[['category', 'categoryCode']]
df_category.drop_duplicates(keep='first', inplace=True)

df_category

<ipython-input-137-c1bd806baa3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category.drop_duplicates(keep='first', inplace=True)


,category,categoryCode
0,Baking,BAKING
1,Beverages,BEVERAGES
4,Candy & Sweets,CANDY_AND_SWEETS
7,Condiments & Sauces,NaN
8,Canned Goods & Soups,NaN
9,Baking,NaN
11,NaN,NaN
13,Magazines,NaN
14,Breakfast & Cereal,NaN
15,Beer Wine Spirits,NaN


It can be seen here that some categories despite having a categoryCode existing hold blank NaN in their catecoryCode column. This data quality issue will be solved by this normalization of separating the CATEGORY entuty from BRAND entity

In [138]:
df_category.sort_values(by='category')

,category,categoryCode
298,Baby,BABY
20,Baby,NaN
0,Baking,BAKING
9,Baking,NaN
19,Beauty,NaN
286,Beauty & Personal Care,NaN
249,Beer Wine Spirits,BEER_WINE_SPIRITS
15,Beer Wine Spirits,NaN
1,Beverages,BEVERAGES
41,Beverages,NaN


In [139]:
# Sorting by both columns so that dropping while keeping first pulls out all existing categoryCodes
# (notice Grocery category before and after)
df_category.sort_values(by=['category', 'categoryCode'], inplace=True)

df_category

<ipython-input-139-c1fd7f4500b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category.sort_values(by=['category', 'categoryCode'], inplace=True)


,category,categoryCode
298,Baby,BABY
20,Baby,NaN
0,Baking,BAKING
9,Baking,NaN
19,Beauty,NaN
286,Beauty & Personal Care,NaN
249,Beer Wine Spirits,BEER_WINE_SPIRITS
15,Beer Wine Spirits,NaN
1,Beverages,BEVERAGES
41,Beverages,NaN


In [140]:
# Dropping the other rows and keeping the ones that have categoryCode
df_category.drop_duplicates(subset='category', keep='first', inplace=True)

# reseting index of CATEGORY table
df_category.reset_index(drop=True, inplace=True)

# Sorting by categoryCode so that thos that do not have categoryCodes go at the bottom
df_category.sort_values(by='categoryCode', inplace=True)

df_category

<ipython-input-140-e5b4ba9c2b25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category.drop_duplicates(subset='category', keep='first', inplace=True)
<ipython-input-140-e5b4ba9c2b25>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category.sort_values(by='categoryCode', inplace=True)


,category,categoryCode
0,Baby,BABY
1,Baking,BAKING
4,Beer Wine Spirits,BEER_WINE_SPIRITS
5,Beverages,BEVERAGES
6,Bread & Bakery,BREAD_AND_BAKERY
8,Candy & Sweets,CANDY_AND_SWEETS
10,Cleaning & Home Improvement,CLEANING_AND_HOME_IMPROVEMENT
13,Dairy & Refrigerated,DAIRY_AND_REFRIGERATED
15,Frozen,FROZEN
16,Grocery,GROCERY


In [141]:
# Dropping the last NaN row
df_category.drop(index=23, axis=0, inplace=True)

df_category

<ipython-input-141-477e5658b4d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category.drop(index=23, axis=0, inplace=True)


,category,categoryCode
0,Baby,BABY
1,Baking,BAKING
4,Beer Wine Spirits,BEER_WINE_SPIRITS
5,Beverages,BEVERAGES
6,Bread & Bakery,BREAD_AND_BAKERY
8,Candy & Sweets,CANDY_AND_SWEETS
10,Cleaning & Home Improvement,CLEANING_AND_HOME_IMPROVEMENT
13,Dairy & Refrigerated,DAIRY_AND_REFRIGERATED
15,Frozen,FROZEN
16,Grocery,GROCERY


In [142]:
df_brands.drop(['category', 'categoryCode'], axis=1, inplace=True)

df_brands

,brand_id,barcode,brand_name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,test brand @1601644365844,NaN,NaN,5f77274dbe37ce6b592e90bf,Cogs
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,53e10d6368abd3c7065097cc,Cogs
1164,5f494c6e04db711dd8fe87e7,511111416173,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,5332fa12e4b03c9a25efd1e7,Cogs
1165,5a021611e4b00efe02b02a57,511111400608,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,5332f5f6e4b03c9a25efd0b4,Cogs


In [143]:
df_brands.isna().sum().sort_values(ascending=True)

,0
brand_id,0
barcode,0
brand_name,0
cpg_id,0
cpg_ref,0
brandCode,234
topBrand,612


In [144]:
df_category.isna().sum().sort_values(ascending=True)

,0
category,0
categoryCode,9


## Receipts

In [145]:
df_receipts

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [146]:
# Extracting receipt_id in clean format
df_receipts['receipt_id'] = df_receipts['receipt_id'].apply(lambda x: x.get('$oid'))

df_receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [147]:
# Extracting all the date columns in Receipts data in datetime format
for i in df_receipts.columns:
    if re.findall(r'date|Date', i):
        df_receipts[i] = df_receipts[i].apply(lambda x: x.get('$date') if str(x) != str(np.nan) else x)
        df_receipts[i] = pd.to_datetime(df_receipts[i]//1000, unit='s')

df_receipts

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27,2021-03-01 10:22:27,NaT,2021-03-01 10:22:28,NaT,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,NaT,2021-03-01 15:42:41,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,NaT,2021-03-01 14:07:37,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,NaT,2021-03-01 13:07:29,NaT,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [148]:
df_receipts.isna().sum().sort_values(ascending=True)

,0
receipt_id,0
createDate,0
dateScanned,0
modifyDate,0
rewardsReceiptStatus,0
userId,0
totalSpent,435
rewardsReceiptItemList,440
purchaseDate,448
purchasedItemCount,484


In [149]:
# Viewing receipt rows that do not have rewardsReceiptItemList
df_receipts[df_receipts.rewardsReceiptItemList.isna()].sort_values(by='receipt_id')

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
71,5ff475820a7214ada10005cf,NaN,NaN,2021-01-05 14:19:46,2021-01-05 14:19:46,NaT,2021-01-05 14:19:46,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5a43c08fe4b014fd6b6a0612
93,5ff5ecb90a7214ada10005f9,NaN,NaN,2021-01-06 17:00:40,2021-01-06 17:00:40,NaT,2021-01-06 17:00:40,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5a43c08fe4b014fd6b6a0612
149,5ff726860a720f05230005ec,NaN,NaN,2021-01-07 15:19:34,2021-01-07 15:19:34,NaT,2021-01-07 15:19:34,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5ff7264e8f142f11dd189504
175,5ff8da570a720f05c5000015,NaN,NaN,2021-01-08 22:19:03,2021-01-08 22:19:03,NaT,2021-01-08 22:19:04,NaT,NaN,NaT,0.0,NaN,REJECTED,0.0,5ff8da28b3348b11c9337ac6
236,5ffce78c0a7214ad4e001045,NaN,NaN,2021-01-12 00:04:27,2021-01-12 00:04:27,NaT,2021-01-12 00:04:27,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,59c124bae4b0299e55b0f330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,603cf2ce0a7217c72c000413,NaN,NaN,2021-03-01 13:57:34,2021-03-01 13:57:34,NaT,2021-03-01 13:57:34,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,NaT,2021-03-01 14:07:37,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,NaT,2021-03-01 15:42:41,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1107,603d28b60a720fde10000445,NaN,NaN,2021-03-01 17:47:34,2021-03-01 17:47:34,NaT,2021-03-01 17:47:34,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33


In [150]:
df_items_list = pd.DataFrame()
df_items_list = df_receipts[['receipt_id', 'rewardsReceiptItemList']]

df_items_list

,receipt_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
1,5ff1e1bb0a720f052300056b,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
2,5ff1e1f10a720f052300057a,"[{'needsFetchReview': False, 'partnerItemId': ..."
3,5ff1e1ee0a7214ada100056f,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
4,5ff1e1d20a7214ada1000561,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
...,...,...
1114,603cc0630a720fde100003e6,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."
1115,603d0b710a720fde1000042a,NaN
1116,603cf5290a720fde10000413,NaN
1117,603ce7100a7217c72c000405,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."


In [151]:
df_receipts.drop('rewardsReceiptItemList', axis=1, inplace=True)

df_receipts

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27,2021-03-01 10:22:27,NaT,2021-03-01 10:22:28,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,NaT,2021-03-01 15:42:41,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,NaT,2021-03-01 14:07:37,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,NaT,2021-03-01 13:07:29,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [152]:
df_receipts.nunique().sort_values(ascending=False)

,0
receipt_id,1119
createDate,1106
dateScanned,1106
modifyDate,1103
finishedDate,552
pointsAwardedDate,523
purchaseDate,358
userId,258
pointsEarned,119
totalSpent,94


Since the receipts dataframe has 1119 rows and there are 1119 unique receipt_ids, there will likely be no duplicate rows. Let's check that.

In [153]:
# Checking duplicate rows in receipts data
df_receipts[df_receipts.duplicated()]

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId


In [154]:
# Checking unique values that 'rewardsReceiptStatus' can take
df_receipts['rewardsReceiptStatus'].unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

In [155]:
# Checking unique values in 'bonusPointsEarnedReason'
df_receipts['bonusPointsEarnedReason'].unique()

array(['Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'All-receipts receipt bonus',
       'Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 3 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 6 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 4 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       nan, 'COMPLETE_PARTNER_RECEIPT', 'COMPLETE_NONPARTNER_RECEIPT'],
      dtype=object)

In [156]:
df_receipts

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27,2021-03-01 10:22:27,NaT,2021-03-01 10:22:28,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,NaT,2021-03-01 15:42:41,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,NaT,2021-03-01 14:07:37,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,NaT,2021-03-01 13:07:29,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


## Items List

In [157]:
df_items_list

,receipt_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
1,5ff1e1bb0a720f052300056b,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
2,5ff1e1f10a720f052300057a,"[{'needsFetchReview': False, 'partnerItemId': ..."
3,5ff1e1ee0a7214ada100056f,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
4,5ff1e1d20a7214ada1000561,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
...,...,...
1114,603cc0630a720fde100003e6,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."
1115,603d0b710a720fde1000042a,NaN
1116,603cf5290a720fde10000413,NaN
1117,603ce7100a7217c72c000405,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."


In [158]:
# Checking content of 'rewardsReceiptItemList' column for first record
df_items_list['rewardsReceiptItemList'][0]

[{'barcode': '4011',
  'description': 'ITEM NOT FOUND',
  'finalPrice': '26.00',
  'itemPrice': '26.00',
  'needsFetchReview': False,
  'partnerItemId': '1',
  'preventTargetGapPoints': True,
  'quantityPurchased': 5,
  'userFlaggedBarcode': '4011',
  'userFlaggedNewItem': True,
  'userFlaggedPrice': '26.00',
  'userFlaggedQuantity': 5}]

In [159]:
# Checking content of 'rewardsReceiptItemList' column for second record
df_items_list['rewardsReceiptItemList'][1]

[{'barcode': '4011',
  'description': 'ITEM NOT FOUND',
  'finalPrice': '1',
  'itemPrice': '1',
  'partnerItemId': '1',
  'quantityPurchased': 1},
 {'barcode': '028400642255',
  'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
  'finalPrice': '10.00',
  'itemPrice': '10.00',
  'needsFetchReview': True,
  'needsFetchReviewReason': 'USER_FLAGGED',
  'partnerItemId': '2',
  'pointsNotAwardedReason': 'Action not allowed for user and CPG',
  'pointsPayerId': '5332f5fbe4b03c9a25efd0ba',
  'preventTargetGapPoints': True,
  'quantityPurchased': 1,
  'rewardsGroup': 'DORITOS SPICY SWEET CHILI SINGLE SERVE',
  'rewardsProductPartnerId': '5332f5fbe4b03c9a25efd0ba',
  'userFlaggedBarcode': '028400642255',
  'userFlaggedDescription': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
  'userFlaggedNewItem': True,
  'userFlaggedPrice': '10.00',
  'userFlaggedQuantity': 1}]

In [160]:
df_items_list['rewardsReceiptItemList'][4]

[{'barcode': '4011',
  'description': 'ITEM NOT FOUND',
  'finalPrice': '1',
  'itemPrice': '1',
  'partnerItemId': '1',
  'quantityPurchased': 1},
 {'barcode': '1234',
  'finalPrice': '2.56',
  'itemPrice': '2.56',
  'needsFetchReview': True,
  'needsFetchReviewReason': 'USER_FLAGGED',
  'partnerItemId': '2',
  'preventTargetGapPoints': True,
  'quantityPurchased': 3,
  'userFlaggedBarcode': '1234',
  'userFlaggedDescription': '',
  'userFlaggedNewItem': True,
  'userFlaggedPrice': '2.56',
  'userFlaggedQuantity': 3}]

In [161]:
df_items_list['rewardsReceiptItemList'][11]

[{'barcode': '013562300631',
  'description': "Annie's Homegrown Organic White Cheddar Macaroni & Cheese Shells, 6 Oz",
  'discountedItemPrice': '50.00',
  'finalPrice': '50.00',
  'itemNumber': '013562300631',
  'itemPrice': '50.00',
  'needsFetchReview': True,
  'needsFetchReviewReason': 'POINTS_GREATER_THAN_THRESHOLD',
  'originalMetaBriteQuantityPurchased': 1,
  'partnerItemId': '1',
  'pointsNotAwardedReason': 'Action not allowed for user and CPG',
  'pointsPayerId': '5332f5f3e4b03c9a25efd0ae',
  'quantityPurchased': 5,
  'rewardsGroup': "ANNIE'S HOMEGROWN MULTI-SERVING MAC & CHEESE",
  'rewardsProductPartnerId': '5332f5f3e4b03c9a25efd0ae'}]

##### DATA QUALITY ISSUE

In [162]:
df_items_list.notna().sum()

,0
receipt_id,1119
rewardsReceiptItemList,679


In [163]:
df_items_list = df_items_list[df_items_list['rewardsReceiptItemList'].notna()]

df_items_list

,receipt_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
1,5ff1e1bb0a720f052300056b,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
2,5ff1e1f10a720f052300057a,"[{'needsFetchReview': False, 'partnerItemId': ..."
3,5ff1e1ee0a7214ada100056f,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
4,5ff1e1d20a7214ada1000561,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
...,...,...
1106,603c7c6c0a7217c72c0003b3,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."
1112,603c3d240a720fde10000373,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."
1113,603cc2bc0a720fde100003e9,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."
1114,603cc0630a720fde100003e6,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."


### Parsing

In [164]:
# Creating a new dataframe called 'df_receipt_items'
# parsing all items in each receipt from the 'df_items' dataframe.
df_receipt_items = pd.DataFrame()

count_failed = 0

for i in df_items_list.index:
    try:
        for json in df_items_list['rewardsReceiptItemList'][i]:
            try:
                temp = pd.DataFrame([json])
                temp['receipt_id'] = df_items_list['receipt_id'][i]
                df_receipt_items = pd.concat([df_receipt_items, temp], ignore_index=True)
            except:
                count_failed += 1
                print("failed to read:", i)
                pass
    except:
        count_failed += 1
        print("failed to read:", i)
        pass

In [165]:
count_failed

0

In [166]:
df_receipt_items

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [167]:
unique_receipts_parsed = df_receipt_items['receipt_id'].nunique()

unique_receipts_parsed

679

In [168]:
# Moving column 'receipt_id' to the first position
col = df_receipt_items.pop('receipt_id')
df_receipt_items.insert(0, 'receipt_id', col)

In [169]:
df_receipt_items.sort_values(by=['receipt_id','barcode'], inplace=True)

df_receipt_items

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
24,5f9c74f70a7214ad07000037,075925306254,NaN,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
25,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,2,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,3,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,4,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,5,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,603d40250a720fde10000459,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6921,603d59e70a7217c72c00045f,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6922,603d59e70a7217c72c00045f,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6919,603d5d6c0a7217c72c000463,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


It can be seen here that there are multiple items within same receipt, each in a new row

In [ ]:
# Checking sparseness of all columns
df_receipt_items.isna().sum().sort_values(ascending=True)

,0
receipt_id,0
partnerItemId,0
finalPrice,174
itemPrice,174
quantityPurchased,174
description,381
discountedItemPrice,1172
originalReceiptItemText,1181
barcode,3851
brandCode,4341


##### DATA QUALITY ISSUE

In [ ]:
rewardGroups_count = df_receipt_items.groupby('barcode')['rewardsGroup'].nunique().explode().reset_index()

rewardGroups_count.sort_values(by='rewardsGroup', inplace=True)

rewardGroups_count

,barcode,rewardsGroup
0,000980000069,0
388,071279275024,0
387,071159020195,0
386,071159001750,0
385,071146002487,0
...,...,...
311,044700033302,1
319,044700073377,1
283,043000081501,1
95,021000045129,2


In [ ]:
rewardGroups_count[rewardGroups_count.rewardsGroup>1]

,barcode,rewardsGroup
95,021000045129,2
409,075925306254,2


In [ ]:
rewardGroups_names = df_receipt_items.groupby('barcode')['rewardsGroup'].unique().explode().reset_index()

rewardGroups_names.sort_values(by='rewardsGroup', inplace=True)

rewardGroups_names

,barcode,rewardsGroup
145,023400022212,AIR WICK ROOM SPRAYS
469,305731769135,ALEVE® LIQUID GELS
470,305731769203,ALEVE® LIQUID GELS
113,021000057832,ANNIE'S HOMEGROWN MICROWAVABLE MAC & CHEESE
138,021000725403,ANNIE'S HOMEGROWN MULTI-SERVING MAC & CHEESE
...,...,...
579,94094,NaN
580,B076FJ92M4,NaN
581,B07BRRLSVC,NaN
582,B08BGBHHP6,NaN


In [ ]:
rewardGroups_names[rewardGroups_names.barcode=='021000045129']

,barcode,rewardsGroup
97,021000045129,ANNIE'S HOMEGROWN MULTI-SERVING MAC & CHEESE
96,021000045129,VELVEETA MACARONI & CHEESE DINNER


In [ ]:
rewardsProductPartnerId_count = df_receipt_items.groupby('barcode')['rewardsProductPartnerId'].nunique().explode().reset_index()

rewardsProductPartnerId_count.sort_values(by='rewardsProductPartnerId', inplace=True)

rewardsProductPartnerId_count

,barcode,rewardsProductPartnerId
567,B08DQDHR2S,0
331,045255148947,0
467,3283,0
468,3332,0
469,3435,0
...,...,...
198,036000495737,1
199,036632011077,1
194,036000432190,1
283,043000081501,1


In [ ]:
rewardsProductPartnerId_names = df_receipt_items.groupby('barcode')['rewardsProductPartnerId'].unique().explode().reset_index()

rewardsProductPartnerId_names.sort_values(by='rewardsProductPartnerId', inplace=True)

rewardsProductPartnerId_names

,barcode,rewardsProductPartnerId
614,884912006806,5332f5f3e4b03c9a25efd0ae
121,021000057832,5332f5f3e4b03c9a25efd0ae
109,021000050765,5332f5f3e4b03c9a25efd0ae
105,021000045129,5332f5f3e4b03c9a25efd0ae
100,021000039340,5332f5f3e4b03c9a25efd0ae
...,...,...
619,94094,NaN
620,B076FJ92M4,NaN
621,B07BRRLSVC,NaN
622,B08BGBHHP6,NaN


In [ ]:
rewardsProductPartnerId_names[rewardsProductPartnerId_names.barcode=='075925306254']

,barcode,rewardsProductPartnerId
458,075925306254,559c2234e4b06aca36af13c6
459,075925306254,5e7cf838f221c312e698a628


In [ ]:
# Delete rows where the value has anomaly
df_receipt_items = df_receipt_items[df_receipt_items['barcode'] != '075925306254']

df_receipt_items

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
25,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,2,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,3,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,4,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,5,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,6,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,603d40250a720fde10000459,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6921,603d59e70a7217c72c00045f,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6922,603d59e70a7217c72c00045f,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6919,603d5d6c0a7217c72c000463,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [ ]:
# Delete rows where the value in column 'B' is 'x'
df_receipt_items = df_receipt_items[df_receipt_items['barcode'] != '021000045129']

df_receipt_items

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
25,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,2,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,3,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,4,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,5,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,5f9c74f70a7214ad07000037,NaN,NaN,NaN,NaN,True,6,True,NaN,034100573065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,603d40250a720fde10000459,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6921,603d59e70a7217c72c00045f,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6922,603d59e70a7217c72c00045f,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6919,603d5d6c0a7217c72c000463,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


# Second: Write queries that directly answer predetermined questions from a business stakeholder

In [ ]:
import sqlite3

# Creating an in-memory SQLite database
connection = sqlite3.connect(':memory:')

# Loading the dataframe as tables in the database
df_users.to_sql('user', connection, index=False)
df_brands.to_sql('brand', connection, index=False)
df_receipts.to_sql('receipt', connection, index=False)

# df_products.to_sql('product', connection, index=False)
df_receipt_items.to_sql('receipt_item', connection, index=False)

print('All tables loaded to database')

All tables loaded to database


### Solution 1

Question: When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [ ]:
query = """
SELECT rewardsReceiptStatus, AVG(totalSpent) AS average_spend
FROM receipt
WHERE rewardsReceiptStatus = 'FINISHED' OR rewardsReceiptStatus = 'REJECTED'
GROUP BY rewardsReceiptStatus
"""

result = pd.read_sql_query(query, connection)

result

,rewardsReceiptStatus,average_spend
0,FINISHED,80.854305
1,REJECTED,23.326056


Answer: average spend from receipts with 'rewardsReceiptStatus’ of ‘Finished’ (meaning same as Accepted) is greater than those having 'rewardsReceiptStatus’ as 'Rejected

### Solution 2

When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [ ]:
query = """
SELECT rewardsReceiptStatus, SUM(purchasedItemCount) AS total_number_of_items_purchased
FROM receipt
WHERE rewardsReceiptStatus = 'FINISHED' OR rewardsReceiptStatus = 'REJECTED'
GROUP BY rewardsReceiptStatus
"""

result = pd.read_sql_query(query, connection)

result

,rewardsReceiptStatus,total_number_of_items_purchased
0,FINISHED,8184.0
1,REJECTED,173.0


Answer: total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Finished’ (meaning same as Accepted) is greater than those with 'rewardsReceiptStatus’ of ‘Rejected’

### Solution 3

Question: Which brand has the most spend among users who were created within the past 6 months?

In [ ]:
# Finding the latest date existing
query = """
SELECT *
FROM receipt
ORDER BY dateScanned DESC;
"""

result = pd.read_sql_query(query, connection)

result

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,603d760e0a720fde1000048e,NaN,None,2021-03-01 23:17:34,2021-03-01 23:17:34,None,2021-03-01 23:17:34,None,NaN,None,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1,603d5d6c0a7217c72c000463,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 21:32:28,2021-03-01 21:32:28,None,2021-03-01 21:32:29,None,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
2,603d59e70a7217c72c00045f,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 21:17:27,2021-03-01 21:17:27,None,2021-03-01 21:17:28,None,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
3,603d40250a720fde10000459,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 19:27:33,2021-03-01 19:27:33,None,2021-03-01 19:27:34,None,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
4,603d30e60a7217c72c00043f,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 18:22:30,2021-03-01 18:22:30,None,2021-03-01 18:22:31,None,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,5fa8d5730a7214adc30001c3,750.0,"Receipt number 1 completed, bonus point schedu...",2020-11-09 05:36:51,2020-11-09 05:36:51,None,2021-01-27 01:13:20,2020-11-09 05:36:52,750.0,2020-11-08 05:36:51,11.0,REJECTED,1.00,5fa8d573770b516dda4f4d21
1115,5fa5ad370a720f05ef000089,750.0,"Receipt number 1 completed, bonus point schedu...",2020-11-06 20:08:23,2020-11-06 20:08:23,2021-01-05 20:53:40,2021-01-05 20:53:40,2021-01-05 20:53:40,9449.8,2020-11-05 20:08:23,11.0,FINISHED,291.00,5fa5ad376a26f611e71ab5ef
1116,5fa5b0ca0a720f05ef0000bf,100.0,"Receipt number 6 completed, bonus point schedu...",2020-11-05 20:23:38,2020-11-05 20:23:38,2021-01-08 21:44:00,2021-01-08 21:44:00,2021-01-08 21:44:00,189.2,2020-11-04 20:23:38,6.0,FINISHED,14.00,5fa5b0b720dc5111dd86dcc1
1117,5f9c74f90a7214ad07000038,300.0,"Receipt number 4 completed, bonus point schedu...",2020-10-30 20:18:01,2020-10-30 20:18:01,2021-01-03 15:39:55,2021-01-03 15:39:55,2021-01-03 15:39:55,389.2,2020-10-29 20:18:01,6.0,FINISHED,14.00,5f9c74e3f1937815bd2c1d73


In [ ]:
query = """
SELECT receipt_item.brandCode, brand.brand_name, SUM(totalSpent) AS total_spend
FROM user
JOIN receipt ON user.user_id = receipt.userId
JOIN receipt_item ON receipt.receipt_id = receipt_item.receipt_id
JOIN brand ON receipt_item.brandCode = brand.brandCode
WHERE user.createdDate >= DATE('2021-03-01', '-6 months')
GROUP BY receipt_item.brandCode
ORDER BY total_spend DESC
LIMIT 5;
"""

result = pd.read_sql_query(query, connection)

result

,brandCode,brand_name,total_spend
0,PEPSI,Pepsi,78870.86
1,KRAFT,Kraft,70858.88
2,KNORR,KNORR,61157.05
3,KLEENEX,Kleenex,56050.44
4,DORITOS,Doritos,48321.46


Answer: 'Pepsi' has the most spend among users who were created within the past 6 months (assuming current date is 1st March 2021)

### Solution 4

Question: Which brand has the most transactions among users who were created within the past 6 months?

In [ ]:
query = """
SELECT receipt_item.brandCode, brand.brand_name, COUNT(receipt.receipt_id) AS total_transactions
FROM user
JOIN receipt ON user.user_id = receipt.userId
JOIN receipt_item ON receipt.receipt_id = receipt_item.receipt_id
JOIN brand ON receipt_item.brandCode = brand.brandCode
WHERE user.createdDate >= DATE('2021-03-01', '-6 months')
GROUP BY receipt_item.brandCode
ORDER BY total_transactions DESC
LIMIT 5;
"""

result = pd.read_sql_query(query, connection)

result

,brandCode,brand_name,total_transactions
0,PEPSI,Pepsi,74
1,KLEENEX,Kleenex,70
2,KNORR,KNORR,60
3,DORITOS,Doritos,55
4,KRAFT,Kraft,28


Answer: 'Pepsi' has the most transactions among users who were created within the past 6 months (assuming current date is 1st March 2021)

# Normalizing REWARDS_ITEM further to 3NF

Since receipt_id and partnerItemId are NOT NULL columns, let's check if they can together form a composite key that can be used as a primary key for the RECEIPT_ITEM entity

In [ ]:
df_receipt_items['composite_key'] = df_receipt_items['receipt_id'] + "_" + df_receipt_items['partnerItemId']
df_receipt_items[df_receipt_items.duplicated(subset='composite_key')]

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,composite_key


Since there are no duplicates when (receipt_id,partnerItemId) is used as key, it can be safely assumed to represent each receipt_item uniquely and hence serve as the primary key

In [ ]:
df_receipt_items.isna().sum().sort_values(ascending=True)

,0
receipt_id,0
partnerItemId,0
composite_key,0
finalPrice,174
itemPrice,174
quantityPurchased,174
description,381
discountedItemPrice,1172
originalReceiptItemText,1181
barcode,3851


In [ ]:
df_receipt_items.nunique().sort_values(ascending=False)

,0
composite_key,6941
description,1889
originalReceiptItemText,1738
partnerItemId,916
finalPrice,828
itemPrice,828
discountedItemPrice,817
receipt_id,679
barcode,568
priceAfterCoupon,334


In [ ]:
df_products = df_receipt_items.drop('composite_key', axis=1)
df_products.drop('receipt_id', axis=1, inplace=True)

df_products

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
25,NaN,NaN,NaN,NaN,True,2,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,True,3,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,True,4,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,True,5,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,True,6,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6921,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6922,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6920,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [ ]:
df_products[df_products.duplicated()]

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
441,034100573065,MILLER LITE 24 PACK 12OZ CAN,1.00,1.00,False,4,True,1.0,034100573065,True,...,NaN,1.0,30.0,77,NaN,1.00,1.00,NaN,NaN,NaN
442,034100573065,MILLER LITE 24 PACK 12OZ CAN,1.00,1.00,False,5,True,1.0,034100573065,True,...,NaN,NaN,30.0,77,NaN,NaN,NaN,NaN,NaN,NaN
440,034100573065,MILLER LITE 24 PACK 12OZ CAN,1.00,1.00,False,3,True,1.0,034100573065,True,...,NaN,1.0,30.0,77,NaN,1.00,1.00,NaN,NaN,NaN
6090,NaN,NaN,NaN,NaN,True,11,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6088,NaN,NaN,NaN,NaN,True,9,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6921,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6922,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6920,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [ ]:
df_products[df_products.duplicated(keep=False)]

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
25,NaN,NaN,NaN,NaN,True,2,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,True,3,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,True,4,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,True,5,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,True,6,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6921,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6922,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6920,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [ ]:
df_products.shape[0]-1228+(1228-919)

5983

In [ ]:
df_products.drop_duplicates(keep='first', inplace=True)

df_products

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
25,NaN,NaN,NaN,NaN,True,2,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,True,3,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,True,4,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,True,5,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,True,6,True,NaN,034100573065,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6861,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6864,665290001184,"Berry Strawberry Conventional, 16 Ounce",3.99,3.99,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.99,NaN
6862,B08BGBHHP6,spigen thin fit designed for iphone 12 mini ca...,12.59,12.59,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.59,NaN


In [ ]:
df_products.isna().sum().sort_values(ascending=True)

,0
partnerItemId,0
finalPrice,34
itemPrice,34
quantityPurchased,34
description,202
discountedItemPrice,755
originalReceiptItemText,759
barcode,3532
brandCode,3624
rewardsProductPartnerId,4059


In [ ]:
l1 = list(df_products.barcode.unique())
l2 = list(df_brands.barcode.unique())
common_elements = np.intersect1d(l1, l2)
common_elements

array(['511111001485', '511111001768', '511111003960', '511111004127',
       '511111101451', '511111104186', '511111104537', '511111204206',
       '511111502142', '511111518044', '511111602118', '511111704140',
       '511111802358', '511111901587', '511111902690', '511111904175'],
      dtype='<U32')

In [ ]:
len(common_elements)

16

In [ ]:
len(l1)

567

In [ ]:
len(l2)

1160

In [ ]:
df_products.isna().sum().sort_values(ascending=True)

,0
partnerItemId,0
finalPrice,34
itemPrice,34
quantityPurchased,34
description,202
discountedItemPrice,755
originalReceiptItemText,759
barcode,3532
brandCode,3624
rewardsProductPartnerId,4059


In [ ]:
unique_product_barcodes = df_products['barcode'].unique()

# Calculate unique values in other columns for each unique ID
result = {}
for id_val in unique_product_barcodes:
    subset = df_products[df_products['barcode'] == id_val]
    result[id_val] = subset.drop('barcode', axis=1).nunique()

# Convert result to DataFrame
df_product_uniqueness = pd.DataFrame(result).T

df_product_uniqueness

,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
034100573065,1,3,3,2,11,1,1,1,1,2,...,0,1,2,1,0,1,1,0,0,0
4011,3,15,15,2,9,1,8,1,1,10,...,1,0,0,0,0,0,0,0,1,0
013562300631,1,1,1,1,1,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
046000832517,1,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B076FJ92M4,1,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
B07BRRLSVC,1,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
665290001184,1,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
B08BGBHHP6,1,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df_product_uniqueness.nunique().sort_values(ascending=False)

,0
partnerItemId,29
itemPrice,15
finalPrice,15
discountedItemPrice,14
originalReceiptItemText,12
priceAfterCoupon,11
pointsEarned,10
quantityPurchased,9
userFlaggedPrice,4
description,4


In [ ]:
df_products = df_products[['barcode', 'preventTargetGapPoints', 'pointsPayerId',
                           'rewardsGroup','rewardsProductPartnerId', 'competitorRewardsGroup',
                           'targetPrice', 'originalFinalPrice', 'competitiveProduct', 'brandCode', 'deleted']]

df_products

,barcode,preventTargetGapPoints,pointsPayerId,rewardsGroup,rewardsProductPartnerId,competitorRewardsGroup,targetPrice,originalFinalPrice,competitiveProduct,brandCode,deleted
25,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6860,B076FJ92M4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6861,B07BRRLSVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6864,665290001184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6862,B08BGBHHP6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_products.sort_values(by=['barcode', 'rewardsProductPartnerId','pointsPayerId', 'brandCode',
                           'rewardsGroup', 'competitorRewardsGroup',
                           'targetPrice', 'originalFinalPrice',
                           'competitiveProduct', 'preventTargetGapPoints', 'deleted'], inplace=True)

df_products

<ipython-input-105-14ea42a3fe86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_products.sort_values(by=['barcode', 'rewardsProductPartnerId','pointsPayerId', 'brandCode',


,barcode,preventTargetGapPoints,pointsPayerId,rewardsGroup,rewardsProductPartnerId,competitorRewardsGroup,targetPrice,originalFinalPrice,competitiveProduct,brandCode,deleted
1934,000980000069,NaN,NaN,NaN,559c2234e4b06aca36af13c6,NaN,NaN,NaN,NaN,NaN,NaN
437,001111132666,NaN,5332f5f6e4b03c9a25efd0b4,DOVE MEN+CARE BODY WASH AND SOAP,5332f5f6e4b03c9a25efd0b4,NaN,NaN,NaN,NaN,BRAND,NaN
467,001111147332,NaN,5332f5f6e4b03c9a25efd0b4,CARESS BODY WASH AND SOAP,5332f5f6e4b03c9a25efd0b4,NaN,NaN,NaN,NaN,BRAND,NaN
2692,005111116022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1940,007874242956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_receipt_items = df_receipt_items[['receipt_id', 'partnerItemId',
                                     'quantityPurchased', 'description',
                                     'itemPrice', 'finalPrice', 'discountedItemPrice','priceAfterCoupon',
                                     'originalReceiptItemText',
                                     'needsFetchReview', 'needsFetchReviewReason',
                                     'pointsEarned', 'pointsNotAwardedReason',
                                     'itemNumber', 'brandCode']]

In [ ]:
df_products

,barcode,preventTargetGapPoints,pointsPayerId,rewardsGroup,rewardsProductPartnerId,competitorRewardsGroup,targetPrice,originalFinalPrice,competitiveProduct,brandCode,deleted
1934,000980000069,NaN,NaN,NaN,559c2234e4b06aca36af13c6,NaN,NaN,NaN,NaN,NaN,NaN
437,001111132666,NaN,5332f5f6e4b03c9a25efd0b4,DOVE MEN+CARE BODY WASH AND SOAP,5332f5f6e4b03c9a25efd0b4,NaN,NaN,NaN,NaN,BRAND,NaN
467,001111147332,NaN,5332f5f6e4b03c9a25efd0b4,CARESS BODY WASH AND SOAP,5332f5f6e4b03c9a25efd0b4,NaN,NaN,NaN,NaN,BRAND,NaN
2692,005111116022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1940,007874242956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_receipt_items

,receipt_id,partnerItemId,quantityPurchased,description,itemPrice,finalPrice,discountedItemPrice,priceAfterCoupon,originalReceiptItemText,needsFetchReview,needsFetchReviewReason,pointsEarned,pointsNotAwardedReason,itemNumber,brandCode
25,5f9c74f70a7214ad07000037,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,USER_FLAGGED,NaN,NaN,NaN,NaN
26,5f9c74f70a7214ad07000037,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,USER_FLAGGED,NaN,NaN,NaN,NaN
27,5f9c74f70a7214ad07000037,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,USER_FLAGGED,NaN,NaN,NaN,NaN
28,5f9c74f70a7214ad07000037,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,USER_FLAGGED,NaN,NaN,NaN,NaN
29,5f9c74f70a7214ad07000037,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,USER_FLAGGED,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,603d40250a720fde10000459,1,1.0,thindust summer face mask - sun protection nec...,11.99,11.99,11.99,11.99,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN
6921,603d59e70a7217c72c00045f,0,1.0,mueller austria hypergrind precision electric ...,22.97,22.97,22.97,22.97,mueller austria hypergrind precision electric ...,NaN,NaN,NaN,NaN,NaN,NaN
6922,603d59e70a7217c72c00045f,1,1.0,thindust summer face mask - sun protection nec...,11.99,11.99,11.99,11.99,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN
6920,603d5d6c0a7217c72c000463,1,1.0,thindust summer face mask - sun protection nec...,11.99,11.99,11.99,11.99,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN
